#Milestone Project 1: Food Vision Big

In [1]:
!nvidia-smi

Wed Sep  8 13:19:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###For mixed precision training we need a GPU of compute capability above 7.0

Tesla K80 is not compatible

Tesla T4 is compatible since it has 7.5 compute capability


In [2]:
!nvidia-smi -L 

GPU 0: Tesla K80 (UUID: GPU-e6c0a458-f509-7512-e5ac-7e289226b075)


##Get helper functions

Import a script of helper functions and load them into our notebook, rather than writing it all

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2021-09-08 13:19:11--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-09-08 13:19:12 (38.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
##Import series of helper functions
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

##Use TensorFlow Datasets to Download Data

In [5]:
#Get TensorFlow Datasets
import tensorflow_datasets as tfds

datasets_list = tfds.list_builders() #List all datasets in TFDS
print("food101" in datasets_list) #Check if food101 dataset is in it

True


#Load in the food101 dataset

In [ ]:
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train","validation"],
                                             shuffle_files=True,
                                             as_supervised=True, #data gets returned in tuple format(data, label)
                                             with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteHS9USX/food101-train.tfrecord


  0%|          | 0/75750 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/food101/2.0.0.incompleteHS9USX/food101-validation.tfrecord


  0%|          | 0/25250 [00:00<?, ? examples/s]

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [ ]:
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [ ]:
class_names = ds_info.features["label"].names #Getting class names
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

In [ ]:
#Let's take one sample of the train data
train_one_sample = train_data.take(1) #Samples are in tuple format (image_tensor, label)

In [ ]:
train_one_sample

<TakeDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>

In [ ]:
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape},
  Image datatype: {image.dtype},
  Target class from Food101 (tensor form):{label},
  Class name(str form): {class_names[label.numpy()]}
  """)

In [ ]:
import tensorflow as tf
tf.reduce_min(image),tf.reduce_max(image)

###Plot image from the Datasets

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False)
print(image.shape)

## Create preprocessing functions for our data

Neural Networks perform best when data is in a certain way(ie., batched, normalized, etc.,)

Not all data comes like this

So we have to write prprocessing functions and map it to your data

Our data:
  * In `unint8` datatype
  * Comprised of all different size tensors
  * Not scaled (not between 0&1 pixel values)

Models like:
  * Data in `float32` (or for mixed precision `float16` and `float32`)
  * For batches, all tensors within a batch to be of same size
  * Scaled

We need to build a function that does this.

There is no need to rescale our data since we're using **EfficientNetBX** model(which has rescaling built-in)

In [ ]:
def preprocess_img(image, label, img_shape=224):
  """
  Converts image datatype from 'uint8' to 'float32' and reshapes image to
  [img_shape,img_shape,3]
  """
  image = tf.image.resize(image, [img_shape,img_shape])
  return tf.cast(image,tf.float32), label #return (preprocessed image,label) tuple

In [ ]:
#Preprocess a single image and check the output
preprocessed_img = preprocess_img(image,label)[0]
print(f"Image before preprocessing: {image[:2]}...,\nShape: {image.shape},\nDatatype:{image.dtype}\n")
print(f"Image after preprocessing: {preprocessed_img[:2]}....,\nShape: {preprocessed_img.shape},\nDatatype: {preprocessed_img.dtype}\n")

#Let's batch and prepare our datasets for our models

We're now going to make our data input pipeline run really fast

In [ ]:
#Map preprocessing function to training(and parallelize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
#Shuffle train data and turn it into batches and prefetch it(load it faster)
train_data = train_data.shuffle(buffer_size=5000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE).cache()

#Map preprocessing function to test data
test_data = test_data.map(map_func=preprocess_img,num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE).cache()


In [ ]:
train_data,test_data

##Create modelling callbacks

We're going to create a couple of callbacks to help us while our model trains
* TensorBoard callback to log training results
* ModelCheckpoint callback to save our models progress after feature extraction

In [ ]:
#Create tensorboard callback (import from helper_functions)
from helper_functions import create_tensorboard_callback

#Create ModelCheckpoint callback to save a models progress during training
checkpoint_path = 'model_checkpoints/cp.ckpt'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_acc",
                                                      dave_best_oly=True,
                                                      save_weights_only=True,
                                                      verbose=0)

#Setup mixed precision Training

In [ ]:
#Turn on mixed precision training (really works only when you have GPU of compute capability 7.0 and greater)
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16") #set global data policy to mixed precision

##Build Feature extraction model

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

#Create base model
input_shape=(224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable=False

#Create functional model
inputs = layers.Input(shape=input_shape, name = "input_layer")
#Note: EfficientNet contains rescaling built-in but if you want to use rescaling in another base_model, use below
# x = preprocessing.Rescaling(1./255)(inputs)

x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax",dtype=tf.float32, name = "softmax_float32")(x)

model = tf.keras.Model(inputs,outputs)

#Compile the model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

In [ ]:
for layer in model.layers:
  print(layer.name, layer.trainable, layer.dtype_policy) 

##Fit the feature extraction model

If your goal is to fine-tune a feature extraction model, the general order is:
  * Build a feature extraction model
  * Fine-tune some of the frozen layers while decreasing lr by atleast 10x

In [ ]:
history_101_feature_extraction = model.fit(train_data,
                                           epochs=3,
                                           steps_per_epoch=len(train_data),
                                           validation_data=test_data,
                                           validation_steps=int(0.15*len(test_data)),
                                           callbacks=[create_tensorboard_callback(dir_name="training_logs",
                                                                                  experiment_name="efficientnerb0_101_feature_extraction"),
                                                      model_checkpoint])

In [ ]:
model.evaluate(test_data)

##Fine-Tuning